In [1]:
import shutil
import sys
from pathlib import Path

import toml

sys.path.append(str(Path("..").resolve()))
from src.pipeline import Config, init_logging
from src.utils.rng import seed

In [ ]:
config_path = Path("config.toml")
config_dict = toml.load(config_path)
config = Config(config_dict)
out_folder = config.out_folder
print(f"Experiment outputs to {out_folder}")

In [ ]:
out_folder.mkdir(parents=True)
shutil.copy("config.toml", out_folder / config_path)
with init_logging(out_folder / "run.log"):
    trainer = config.to_trainer()
    seed(42)
    trainer.train()